Our first presentation was going to answer the question, does healthy equate to happy?  We had great ideas in regards to the data we wanted to collect, as the first go around on the optional data sets and API's appeared to have the data we were looking to measure. Upon further investigatioin,  the step counts, physical stats and gender information was a bit harder to pull from an API than we thought.

Hence, we are now working to see if we can use data provided by the census API, a Kaggle data series that contained over 2,000 Chipotle stores as well as a sample set of data from Kaggle regarding other fast food locations, to see if we can predict what factors may influence a Chipotle location. Originally we had a few different years of information, but because we could not obtain the 2019 census data, we had enough with 2018 we used that year as our basis.

Kaggle data: https://www.kaggle.com/jeffreybraun/chipotle-locations
Sample data from other fast food locations: https://www.kaggle.com/datafiniti/fast-food-restaurants

First we pulled and cleaned all of the data.

In [ ]:
import pandas as pd
import numpy as np
import requests
from census import Census

# Census API Key
from config import api_key
c = Census(api_key, year=2018)

## Census Data

In [ ]:
# Run Census Search to retrieve data on all zip codes (2017 ACS5 Census)
# See: https://github.com/CommerceDataService/census-wrapper for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels
census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E"), {'for': 'zip code tabulation area:*'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "NAME": "Name", "zip code tabulation area": "Zipcode"})

In [ ]:
# Final DataFrame
census_pd = census_pd[["Zipcode", "Population", "Median Age", "Household Income",
                       "Per Capita Income", "Poverty Count"]]

# Visualize
print(len(census_pd))
census_pd.head()

In [ ]:
# Add Poverty Rate at the end
census_pd["Poverty Rate"]=census_pd["Poverty Count"]*100/ census_pd["Population"]


In [ ]:
# Drop all NaN data
census_pd.dropna(subset=['Household Income', 'Per Capita Income', 'Poverty Rate'], inplace=True)
census_pd.info()

We would need latitude and longitude information for a map later referenced.  A pull from the google places API was not working, and I did not want to run up my API account so I used the referenced link to obtain the information I could then add into the census data. https://public.opendatasoft.com/explore/dataset/us-zip-code-latitude-and-longitude/table/

In [ ]:
# Add Latitude and Longitude to census data for later use
latlong ="Resources/us-zip-code-latitude-and-longitude.csv"
# Read files into Pandas DF
latlong= pd.read_csv(latlong)

In [ ]:
print(len(latlong))
latlong.head()
latlong.columns

In [ ]:
# There was an additional space in the Zipcode series from the latitidue file so it had to be renamed prior to merging
latlong=latlong.rename(columns={'Zipcode ': 'Zipcode'})
latlong.head(2)

In [ ]:
# merge with census data
clean_2018_census_pd = pd.merge(census_pd, latlong,on='Zipcode')
clean_2018_census_pd.head(2)

In [ ]:
# made all Zipcodes integers instead of floats
clean_2018_census_pd['Zipcode']= clean_2018_census_pd['Zipcode'].astype(int)

In [ ]:
# rename columns for easier merging with restaurant data
clean_2018_census_pd = clean_2018_census_pd[['Zipcode','Latitude', 'Longitude','Population', 'Median Age', 'Household Income', 'Per Capita Income', 'Poverty Count', 'Poverty Rate']]
clean_2018_census_pd

In [ ]:
# export to CSV
clean_2018_census_pd.to_csv("Resources/census_data_clean_2018.csv", encoding="utf-8", index=False)

## Restaurant Data

In [ ]:
# load all restaurant first and clean
fast_food_restaurants_2018 = "../Resources/Fast_Food_Restaurants_2018.csv"
# Read files into Pandas DF
Rest_2018 = pd.read_csv(fast_food_restaurants_2018)

In [ ]:
# remove columns we will not need
Rest_2018=Rest_2018.drop(['id','sourceURLs','websites'], axis =1)
Rest_2018.columns

In [ ]:
# reorganize/rename columns
Rest_2018 = Rest_2018.rename(columns={'name':'Name', 'address': 'Address', 'city': 'City', 'country': 'Country', 'latitude': 'Latitude', 'longitude':'Longitude', 'postalCode': 'Zip Code',\
                                     'province': 'State'})
Rest_2018 = Rest_2018[['Name', 'Address', 'City','State', 'Zip Code','Country', 'Latitude', 'Longitude']]
Rest_2018.head()

In [ ]:
# get unique names in list and sort alphabetically to assist with cleaning
names = Rest_2018['Name'].unique()
names.sort()
print(len(names))
print(names)

In [ ]:
# Rename all locations that were duplicate but had different capitalizations, punctuations, etc
Rest_2018['Name'] = Rest_2018['Name'].replace({"Arby's - Closed":"Arby's",'Arbys': "Arby's", "Auntie Anne's": "Auntie Anne's Pretzels","Baker's Drive Thru":"Baker's Drive-thru","Ben and Jerry's": "Ben & Jerry's",'Bob Evans Restaurant':'Bob Evans',"Bojangles' Famous Chicken 'n Biscuits": 'Bojangles', 'Burger King¬Æ': 'Burger King','Capri Italian Restaurant':'Capri Restaurant',"Carl's Jr.":"Carl's Jr","Carl's Jr. / Green Burrito":"Carl's Jr","Carl's Jr / Green Burrito":"Carl's Jr",'Caseys Carry Out Pizza':"Casey's General Store","Charley's Grilled Subs":'Charleys Philly Steaks',"Checker's Pizza":'Checkers','Chick-fil-A':'Chick-Fil-A', 'Chipotle':'Chipotle Mexican Grill','Dairy Queen (Treat Only)':'Dairy Queen','Dunkin Donuts':"Dunkin' Donuts",'Five Guys Burgers Fries':'Five Guys', 'Five Guys Burgers And Fries':'Five Guys',"Foster's Freeze":'Fosters Freeze', "Hardee's":'Hardees',"Hardee's / Red Burrito":'Hardees', "Hardee's/red Burrito":'Hardees','Jack in the Box':'Jack In The Box', 'Jack in the Box -':'Jack In The Box'})
Rest_2018['Name'] = Rest_2018['Name'].replace({"Jimmy John's":'Jimmy Johns','KFC - Kentucky Fried Chicken':'KFC','KFC/AW':'KFC','KFC/Long John Silvers':'KFC', 'KFC/Taco Bell':'KFC'})
Rest_2018['Name'] = Rest_2018['Name'].replace({"Little Caesar's Pizza":'Little Caesars', 'Little Caesars Pizza':'Little Caesars',"Long John Silver's / AW" 'Long John Silvers':"Long John Silver's",'Long John Silvers / A&W':"Long John Silver's",'Mc Donalds':"McDonald's","McDonald's of Rolesville":"McDonald's",'McDonalds':"McDonald's","McDonalds's":"McDonald's","Mcdonald's":"McDonald's",'Mcdonalds':"McDonald's",'Mcdonalds Whitehouse':"McDonald's"})
Rest_2018['Name'] = Rest_2018['Name'].replace({'Panda Express Innovation Kitchen':'Panda Express' ,'PepperJax Grill':'Pepperjax Grill',"Popeye's Louisiana Kitchen":'Popeyes','Popeyes Chicken & Biscuits':'Popeyes','Popeyes Chicken Biscuits':'Popeyes','Popeyes Louisiana Kitchen':'Popeyes'})
Rest_2018['Name'] = Rest_2018['Name'].replace({'QDOBA Mexican Eats':'Qdoba Mexican Grill', 'Qdoba Mexican Eats':'Qdoba Mexican Grill', "Quizno's": 'Quiznos', 'Quiznos Sub':'Quiznos',"Raising Cane's":'Raising Canes', "Raising Cane's Chicken Fingers":'Raising Canes'})
Rest_2018['Name'] = Rest_2018['Name'].replace({"Rally's Hamburgers":"Rally's", 'Rallys':"Rally's",'Roma Pizza':'Romas Pizza','SONIC Drive In': 'SONIC Drive-In', 'SUBWAY¬Æ':'SUBWAY'})
Rest_2018['Name'] = Rest_2018['Name'].replace({'Sonic':'SONIC Drive-In', "Sonic America's Drive-In":'SONIC Drive-In','Sonic Drive In':'SONIC Drive-In','Sonic Drive in':'SONIC Drive-In','Sonic Drive-In':'SONIC Drive-In',"Steak 'n Shake":'Steak N Shake','Taco Bell / KFC':'Taco Bell' ,'Taco Bell/KFC':'Taco Bell','Taco Bell/Pizza Hut':'Taco Bell'})
Rest_2018['Name'] = Rest_2018['Name'].replace({ 'Wienerschitzel':'Wienerschnitzel','Wingstop Restaurant':'Wingstop',"Wolf's Dairy Queen" :'Dairy Queen',"Zaxby's Chicken Fingers & Buffalo Wings": "Zaxby's",'b.good':'B.GOOD', 'A&W/Long John Silvers':"Long John Silver's"})
# After going through team decided to rename Pizza Hut, Taco Bell and KFC to YUM Brands
Rest_2018['Name'] = Rest_2018['Name'].replace({'Pizza Hut': 'YUM Brands', 'KFC': 'YUM Brands','Taco Bell': 'YUM Brands'})

In [ ]:
print(len(Rest_2018))

In [ ]:
# Remove locations that are not food related 'Walmart Supercenter''T-Mobile''Hilton Boston Logan Airport''7-Eleven'
# https://thispointer.com/python-pandas-how-to-drop-rows-in-dataframe-by-conditions-on-column-values/
indexNames= Rest_2018[(Rest_2018['Name']=='Walmart Supercenter')].index
Rest_2018.drop(indexNames, inplace=True)

In [ ]:
indexNames= Rest_2018[(Rest_2018['Name']=='T-Mobile')].index
Rest_2018.drop(indexNames, inplace=True)

In [ ]:
indexNames= Rest_2018[(Rest_2018['Name']=='Hilton Boston Logan Airport')].index
Rest_2018.drop(indexNames, inplace=True)

In [ ]:
indexNames= Rest_2018[(Rest_2018['Name']=='7-Eleven')].index
Rest_2018.drop(indexNames, inplace=True)
print(len(Rest_2018))
Rest_2018['Name'].nunique()

In [ ]:
# find number of entries for each location
Rest_2018['Name'].value_counts()

In [ ]:
# Remove all locations that have less than 50 entries
# https://www.thetopsites.net/article/58467286.shtml
threshold = 50
clean_rest_2018 = Rest_2018[Rest_2018.groupby('Name')['Name'].transform('count')>threshold].copy()
print(len(clean_rest_2018))
clean_rest_2018['Name'].value_counts()

In [ ]:
# export to csv as this is clean data incase anything happens while added in other informatoin
clean_rest_2018.to_csv('../Resources/clean_rest_2018.csv', encoding='utf-8', index = False)

In [ ]:
# combine with chipotle csv
# first read get file then read into pandas DF
Chipotle_stores = "../Resources/chipotle_stores.csv"
chipotle = pd.read_csv(Chipotle_stores)
chipotle.head(2)
print(len(chipotle))

In [ ]:
clean_rest_2018.head(2)
print(len(clean_rest_2018))

In [ ]:
# merge 2 restaurant dataframes. Used concat to just add the other restaurant data underneath the chipotle information
all_rest_df = pd.concat([chipotle, clean_rest_2018])
print(len(all_rest_df))
all_rest_df

In [ ]:
# Add census data to right of all rest data(printed head to get column names)
census2_2018.head(2)

### The next three cells were from my tutor 
https://github.com/ealong/Springboard-Capstone1/blob/master/capstone1-wrangling.ipynb

In [ ]:
# Converted to string because restaurant data information was listed as string and could not merge properly
# Adjusted series name to allow proper merge as well
# Confirm number of zip codes with less than 5 digits
census2_2018['Zipcode'] = census2_2018['Zipcode'].astype(str)
census_2018 = census2_2018.rename(columns={'Zipcode': 'Zip Code'})
census_2018_badzip= census_2018[census_2018['Zip Code'].str.len()<5]
print(census_2018_badzip)

In [ ]:
#Research confirms that for the cities and states with zip codesless than 5 digits, the zip codes begin with 0.
#Excel truncates the leading zero. (#e.g. 06001 becomes 6001 in Excel) 

def format_postal_codes(row):
    zipcode= row['Zip Code']
    return zipcode[:5].zfill(5)

In [ ]:
# Fix zip codes to have 5 digits
census_2018['Zip Code'] = census_2018.apply(format_postal_codes, axis=1)

census_2018_badzip = census_2018[census3_2018['Zip Code'].str.len()<5]
print(census_2018_badzip)

In [ ]:
# Add census data to right of all rest data
all_rest_new = pd.merge(all_rest_df,census_2018, on="Zip Code", how='left')
all_rest_new.head(50)

In [ ]:
# Remove and rename extra lat/long columns
all_rest_new = all_rest_new.rename(columns={'Latitude_x': 'Latitude', 'Longitude_x':'Longitude'})
all_rest_df = all_rest_new[['Name', 'Address', 'City','State', 'Zip Code','Country', 'Latitude', 'Longitude','Population', 'Median Age', 'Household Income','Per Capita Income', 'Poverty Count', 'Poverty Rate']]
all_rest_df.tail(50)

In [ ]:
# Export to CSV for others to use
all_rest_df.to_csv("Resources/All_rest_2018.csv", encoding="utf-8", index=False)

## Stock Data Cleaning